# Age Validation
This script is to extract age, validate it with the CiC dataset and to create an age started care.

In [ ]:
# Load libraries
library(dplyr)
library(here)
library(ggplot2)
library(lubridate)
library(bigrquery)
bq_auth()

In [ ]:
# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

# Store Tables of Interest
targetdb1 <-'yhcr-prd-phm-bia-core.CB_FDM_ChildrensSocialCare'
targetdb1 <-gsub(' ','',targetdb1)
print (targetdb1)

targetdb2 <-'yhcr-prd-phm-bia-core.CB_FDM_MASTER'
targetdb2 <-gsub(' ','',targetdb2)
print (targetdb2)


As there are multiple care entries the minimum startdate will be extracted. This assumes that the people it is the first time that people in this dataset entered care. I will first filter out where the partial postcode is bradford so that the minimum startdate is not related to a different area.  


In [ ]:
# Create SQL command

sql1 <- paste('
WITH distinct_pi AS (
  SELECT person_id, min(StartDate) as StartDate
  FROM ', targetdb1, '.tbl_CiC
  WHERE PCArea_Home LIKE \'BD%\'
  GROUP BY person_id
)
SELECT distinct a.person_id, a.StartDate, c.YearOfBirth, b.year_of_birth, b.month_of_birth
FROM distinct_pi a
JOIN ', targetdb1, '.tbl_CiC c ON a.person_id = c.person_id AND a.StartDate = c.StartDate
JOIN ', targetdb2,'.person b ON b.person_id = a.person_id
LIMIT 4000;
', sep = "")


#This runs it
tb3 <- bq_project_query(project_id, sql1)

#This loads  it into an R data frame
table <- bq_table_download(tb3)
#This displays it
table


In [ ]:
# Check if years match 
matchyear <- table %>%
 filter(YearOfBirth == year_of_birth)
nrow(matchyear)

The year of birth matches in both datasets.

In [ ]:
# Combine year_of_birth and month_of_birth into a date variable
table2 <- table %>%
  mutate(date_of_birth = make_date(year_of_birth, month_of_birth, 15))
         
head(table2)

In [ ]:
# Create age started care variable
table3 <- table2 %>%
  mutate(
    age_start = floor(as.numeric(difftime(StartDate, date_of_birth, units = "weeks")) / 52.25
  ))
head(table3)

In [ ]:
# Create age_start_bands variable
table4 <- table3 %>%
  mutate(age_start_bands = cut(age_start, 
                               breaks = c(-Inf, 2, 4, 7, 11, 14, Inf),
                               labels = c("0-2", "3-4", "5-7", "8-11", "12-14", "15-18")))
                               
head(table4)


The age bands can be changed if there are more appropriate age bands such as developmental categories or any findings in the data.

In [ ]:
# Create table to upload
upload <- table4 %>%
 select(person_id, date_of_birth)

schema <- list(
  list("person_id", "INTEGER"),
  list("date_of_birth", "DATE")
)

# Reverse the column order
reverse <- upload[, rev(colnames(upload))]
bq_table_upload("yhcr-prd-phm-bia-core.CB_2353.CiC_birth_date", reverse, schema = schema)

In [ ]:
# Create Year_entered_care variable
table5 <- table4 %>%
  mutate(year_entered_care = year(StartDate))
head(table5)

In [ ]:
# Create table to upload
upload <- table5 %>%
 select(person_id, StartDate, date_of_birth, age_start, age_start_bands, year_entered_care)

# Change structure
upload$age_start_bands <- as.character(upload$age_start_bands)
upload$year_entered_care <- as.numeric(upload$year_entered_care)

schema <- list(
  list("person_id", "INTEGER"),
  list("StartDate", "DATE"),
  list("date_of_birth", "DATE"),
  list("age_start", "INTEGER"),
  list("age_start_bands", "STRING"),
  list("year_entered_care", "INTEGER")
)

# Reverse the column order
reverse <- upload[, rev(colnames(upload))]
bq_table_upload("yhcr-prd-phm-bia-core.CB_2353.CiC_age_start_care", reverse, schema = schema)